# Utilizando o banco de dados raspado do site: https://genealogy.math.ndsu.nodak.edu

O banco de dados é composto por três tabelas do SQLite, organizando as informações da seguinte forma:

- researchers: Armazena o ID e o nome de cada pesquisador.
- academic_titles: Registra os detalhes de cada título acadêmico, vinculando-os a um pesquisador por meio do seu ID.
- advisors_academic_titles: Mapeia a relação entre orientadores e títulos acadêmicos, que, por sua vez, estão associados a pesquisadores.

## Bibliotecas:

In [ ]:
#!pip install faiss-cpu
#!pip install pyoperon
#!pip install gplearn

In [ ]:
import sqlite3
import json
#import pickle   # Armazenar e carregar modelo treinado
#import faiss
import os
import joblib
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from dotenv import load_dotenv
from collections import defaultdict
from openai import OpenAI, ChatCompletion
from pyoperon.sklearn import SymbolicRegressor
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, normalize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
#from getpass import getpass

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Caminho onde o banco está/será salvo
db_sqlite = "./data/mgp.sqlite"

## Objetos:

In [ ]:
# Objeto pesquisador
class researcher:
  def __init__(self, researcher_id, name, academic_titles, descendants_id):
    self.researcher_id = researcher_id
    self.name = name
    self.academic_titles = academic_titles
    self.descendants_id = descendants_id

  def __str__(self):
    return f'{self.researcher_id} - {self.name} - {self.academic_titles} - {self.descendants_id}'

  def to_list(self):
    academic_titles_list = [academic_title.to_list() for academic_title in self.academic_titles]
    return [self.researcher_id, self.name, academic_titles_list, self.descendants_id]

  def save_to_db(self, db_sqlite):
    with sqlite3.connect(db_sqlite) as conn:
      cursor = conn.cursor()
      conn = sqlite3.connect(db_sqlite)
      cursor = conn.cursor()

      # Cria as tabelas se não existir
      cursor.execute('''
        CREATE TABLE IF NOT EXISTS researchers (
          researcher_id INTEGER PRIMARY KEY,
          name TEXT NOT NULL
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS academic_titles (
          academic_title_id INTEGER PRIMARY KEY,
          researcher_id INTEGER NOT NULL,
          title TEXT NOT NULL,
          dissertation TEXT,
          institution TEXT,
          year INTEGER,
          country TEXT,
          FOREIGN KEY (researcher_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS advisors_academic_titles (
          advisor_id INTEGER NOT NULL,
          academic_title_id INTEGER NOT NULL,
          PRIMARY KEY (advisor_id, academic_title_id),
          FOREIGN KEY (advisor_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE,
          FOREIGN KEY (academic_title_id) REFERENCES academic_titles (academic_title_id) ON DELETE CASCADE
        )
      ''')

      # Inserindo os dados
      cursor.execute('''
        INSERT INTO researchers (researcher_id, name)
        VALUES (?, ?)
      ''', (self.researcher_id, self.name))

      for academic_title in self.academic_titles:
        cursor.execute('''
          INSERT INTO academic_titles (researcher_id, title, dissertation, institution, year, country)
          VALUES (?, ?, ?, ?, ?, ?)
        ''', (self.researcher_id, academic_title.title, academic_title.dissertation, academic_title.institution, academic_title.year, academic_title.country))
        academic_title_id = cursor.lastrowid
        if isinstance(academic_title.advisors_id, (list, tuple)):
          for advisor_id in academic_title.advisors_id:
            cursor.execute('''
              INSERT INTO advisors_academic_titles (advisor_id, academic_title_id)
              VALUES (?, ?)
            ''', (advisor_id, academic_title_id))

      conn.commit()

  def close_db(db_sqlite):
    conn = sqlite3.connect(db_sqlite)
    conn.close()

# Objeto título acadêmico
class academic_title:
  def __init__(self, title, dissertation, institution, year, country, advisors_id):
    self.title = title
    self.dissertation = dissertation
    self.institution = institution
    self.year = year
    self.country = country
    self.advisors_id = advisors_id

  def __str__(self):
    return f'{self.title} - {self.dissertation} - {self.institution} - {self.year} - {self.country} - {self.advisors_id}'

  def to_list(self):
    return [self.title, self.dissertation, self.institution, self.year, self.country, self.advisors_id]


class researcher_numeric_attributes:
  def __init__(self, researcher_dissertations, advisor_ids, student_ids):
    '''Retorna os atributos numéricos do pesquisador'''

    self.num_academic_titles = len(researcher_dissertations)
    self.first_year = min([diss[1] for diss in researcher_dissertations])
    self.last_year = max([diss[1] for diss in researcher_dissertations])
    self.num_advisors = len(advisor_ids)
    self.num_students = len(student_ids)
    self.average_years = average_between_years([diss[1] for diss in researcher_dissertations])

  def __str__(self):
    return f'{self.num_academic_titles} - {self.first_year} - {self.last_year} - {self.num_advisors} - {self.num_students} - {self.average_years}'

## Funções:

In [ ]:
def search_db(db_sqlite, query, params=()):
  '''Essa função usa query e parâmetros para buscar no banco'''

  with sqlite3.connect(db_sqlite) as conn:
        cursor = conn.cursor()
        cursor.execute(query, params)

        return cursor.fetchall()


def generate_sql_query(user_query):
    '''Essa função usa IA para converter pergunta em SQL parametrizado.'''

    system_prompt = """Você é um assistente especializado em bancos de dados SQLite. Converta perguntas em SQL parametrizado usando `?` para os valores.

    Estrutura do banco que você é especialista:
    researchers (
      researcher_id INTEGER PRIMARY KEY,
      name TEXT NOT NULL
    )

    academic_titles (
      academic_title_id INTEGER PRIMARY KEY,
      researcher_id INTEGER NOT NULL,
      title TEXT NOT NULL,
      dissertation TEXT,
      institution TEXT,
      year INTEGER,
      country TEXT,
      FOREIGN KEY (researcher_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE
    )

    advisors_academic_titles (
      advisor_id INTEGER NOT NULL,
      academic_title_id INTEGER NOT NULL,
      PRIMARY KEY (advisor_id, academic_title_id),
      FOREIGN KEY (advisor_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE,
      FOREIGN KEY (academic_title_id) REFERENCES academic_titles (academic_title_id) ON DELETE CASCADE
    )

    Exemplo de saída correta:
    ['SELECT name FROM researchers WHERE researcher_id = ?', ['9', '22']]

    Retorne apenas a SQL e os valores em uma lista Python, sem explicações. Não inclua comandos DELETE ou UPDATE.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ]
    )

    return eval(response.choices[0].message.content.strip())


def get_advisor_and_student_ids(researcher_id):
  '''Essa função retorna a lista de orientadores e alunos de um pesquisador'''

  # Busca todos os orientadores (advisor_id) que estão associados a títulos acadêmicos pertencentes a um pesquisador específico (researcher_id).
  advisor_ids = search_db(db_sqlite,
                          "SELECT aat.advisor_id\
                           FROM advisors_academic_titles aat\
                           JOIN academic_titles at ON aat.academic_title_id = at.academic_title_id\
                           WHERE at.researcher_id = ?", (researcher_id,))

  # Busca os IDs dos pesquisadores que têm pelo menos um título acadêmico orientado pelo pesquisador especificado (researcher_id)
  student_ids = search_db(db_sqlite,
                          "SELECT r.researcher_id\
                           FROM researchers r\
                           JOIN academic_titles at ON r.researcher_id = at.researcher_id\
                           JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id\
                           WHERE aat.advisor_id = ?", (researcher_id,))

  # Corrige a lista
  advisor_ids = [row[0] for row in advisor_ids]
  student_ids = [row[0] for row in student_ids]

  return advisor_ids, student_ids

def list_to_id_and_name(list_id):
  '''Essa função retorna uma lista com ID e nome de uma lista de ID'''

  list_id_and_name = search_db(db_sqlite,
                                "SELECT researcher_id, name\
                                 FROM researchers\
                                 WHERE researcher_id IN ({})".format(",".join("?" * len(list_id))),
                                tuple(list_id))

  return list_id_and_name


def get_dissertations_researcher_son(db_sqlite, researcher_id):
  '''Retorna dissertações do pesquisador e de seus alunos'''
  with sqlite3.connect(db_sqlite) as conn:
      cursor = conn.cursor()

      # Dissertações e ano do próprio pesquisador
      cursor.execute('''
          SELECT dissertation, year FROM academic_titles
          WHERE researcher_id = ?
      ''', (researcher_id,))
      researcher_dissertations = [row for row in cursor.fetchall()]

      # Dissertações e ano da dissertações dos alunos (alunos desse pesquisador são os que têm ele como orientador)
      cursor.execute('''
          SELECT at.dissertation, year FROM academic_titles at
          JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id
          WHERE aat.advisor_id = ?
      ''', (researcher_id,))
      students_dissertations = [row for row in cursor.fetchall()]

  return researcher_dissertations, students_dissertations


def average_between_years(list_year):
  if len(list_year) > 1:
    return np.median([list_year[i] - list_year[i - 1] for i in range(1, len(list_year))])
  else:
    return 0


# Aplicando a transformação aos inputs categóricos
def transformar_inputs_categoricos(label_encoders, researcher_new):
    for col in ["academic_title", "institution", "country"]:
        researcher_new[col] = researcher_new[col].strip().lower().replace(" ", "")
        if researcher_new[col] in label_encoders[col].classes_:
            researcher_new[col] = int(label_encoders[col].transform([researcher_new[col]])[0])
        else:
            print(f"Aviso: '{researcher_new[col]}' não está nos dados de treinamento. Usando valor padrão -1.")
            researcher_new[col] = -1  # Valor para desconhecidos
    return researcher_new

## Consultas sem uso de ferramentas:

In [ ]:
# Busca toda a base de dados

df_all = pd.read_sql_query('''SELECT
                        r.researcher_id,
                        r.name AS researcher_name,
                        at.academic_title_id,
                        at.title AS academic_title,
                        at.dissertation,
                        at.institution,
                        at.year,
                        at.country,
                        GROUP_CONCAT(aat.advisor_id) AS advisor_ids  -- Lista os orientadores em uma única coluna
                    FROM academic_titles at
                    JOIN researchers r ON r.researcher_id = at.researcher_id
                    LEFT JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id
                    GROUP BY at.academic_title_id''', sqlite3.connect(db_sqlite))
sqlite3.connect(db_sqlite).close()

print("Banco sem otimização gerado pela raspagem:")
df_all.tail(10)

In [ ]:
# Busca orientadores e alunos de um pesquisador
researcher_id = 9#16

print(f'🔎 Buscando informações do pesquisador {researcher_id}...\n')
# Busca ID de todos os orientadores e alunos de um pesquisador
advisor_ids, student_ids = get_advisor_and_student_ids(researcher_id)


if advisor_ids:
    print(f'🎓 IDs dos orientadores encontrados: {advisor_ids}')

    # Busca o ID e o nome dos pesquisadores cujos researcher_id estão na lista de advisor_ids
    advisors = list_to_id_and_name(advisor_ids)
    print("📋 Lista de orientadores:")
    for advisor_id, advisor_name in advisors:
        print(f'🔹 ID: {advisor_id}, Nome: {advisor_name}')
else:
    print("❌ Nenhum orientador encontrado.")


print("\n" + "="*50 + "\n")   # Passa uma linha no terminal


if student_ids:
    print(f'🎓 IDs dos alunos encontrados: {student_ids}')

    # Busca os nomes e IDs dos pesquisadores cujos IDs estão na lista student_ids.
    students = list_to_id_and_name(student_ids)
    print("📋 Lista de alunos:")
    for student_id, student_name in students:
        print(f'🔹 ID: {student_id}, Nome: {student_name}')
else:
    print("❌ Nenhum aluno encontrado.")

In [ ]:
# Busca dissertações do pesquisador e de seus alunos
researcher_dissertations, students_dissertations = get_dissertations_researcher_son(db_sqlite, researcher_id)

print("Dissertações do pesquisador:",[diss[0] for diss in researcher_dissertations])
print("Dissertações dos alunos:", [diss[0] for diss in students_dissertations])

In [ ]:
print('Atributos númericos\n')
researcher_attributes = researcher_numeric_attributes(researcher_dissertations, advisor_ids, student_ids)

print(
      'Número de títulos acadêmicos:', researcher_attributes.num_academic_titles,
      '\nAno do primeiro e do último título:', researcher_attributes.first_year, 'e', researcher_attributes.last_year,
      '\nNúmero de orientadores:', researcher_attributes.num_advisors,
      '\nNúmero de alunos orientados:', researcher_attributes.num_students,
      '\nTempo médio entre seus títulos:', researcher_attributes.average_years,
      '\n\n',
      '\bTotal de pesquisadores:', search_db(db_sqlite, 'SELECT COUNT(*) FROM researchers')[0][0],
      '\nTotal de orientadores:', search_db(db_sqlite, 'SELECT COUNT(DISTINCT advisor_id) FROM advisors_academic_titles')[0][0],
      '\nTotal de não orientadores:', search_db(db_sqlite, 'SELECT COUNT(*) FROM researchers WHERE researcher_id NOT IN (SELECT advisor_id FROM advisors_academic_titles)')[0][0],
      )

# Utilizando inteligência artificial (Não implementado):

In [ ]:
# Pedir a chave sem exibi-la
#os.environ["OPENAI_API_KEY"] = getpass("Digite sua chave de API: ")

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def chatbot(user_query):
    sql_data = generate_sql_query(user_query)
    print("🔍 Query gerada:", sql_data)

    try:
        results = search_db(db_sqlite, sql_data[0], sql_data[1])
        if results:
            return results
        return "Nenhum resultado encontrado."
    except Exception as e:
        return f"Erro ao executar a query: {e}"

# Testando
user_input = "Qual o total são orientadores"
print(chatbot(user_input))

# Utilizando Regressão Simbólica (PyOperon)

In [ ]:
### Mostrando o banco otimizado com colunas de irmãos acadêmicos e geração

df_optimized = pd.read_feather("./data/mgp_optimized.feather")
#df_optimized.to_csv("./data/csv/mgp_optimized.csv", index=False)

print("Mostrando banco otimizado. Colunas com listas python corrigida, e colunas de irmãos acadêmicos e geração geradas:")
df_optimized[256:261]

In [ ]:
### Configurando e tratando os dados do banco

df_optimized = pd.read_feather("./data/mgp_optimized.feather")

# Definindo as colunas relevantes para prever o número de alunos
features = ["academic_title", "institution", "year", "country", "num_advisors", "num_siblings", "generation"]
target = "num_students"
print("Dados usados para predizer o número de alunos de um orientador:\n", features)

#df_optimized = df_optimized.head(300)

# Remover valores nulos e se quiser valores com zero
print("Tamanho do banco com valores nulos:", len(df_optimized))
numeric_columns = df_optimized.select_dtypes(include=['number']).columns
df_optimized = df_optimized.dropna(subset=numeric_columns)
df_optimized = df_optimized[(df_optimized[numeric_columns] >= 0).all(axis=1)]   # Colocar 1 aqui se quiser remover os valores zero
df_optimized = df_optimized[df_optimized['year'] > 1980]
df_optimized = df_optimized[df_optimized['year'] < 2024]
print("Novo tamanho do banco sem valores nulos:", len(df_optimized))

## Existe uma relação entre o número de alunos orientados por um pesquisador e características como instituição, país, ano do título, número de orientadores, números de irmão acadêmicos e posição geracional?

### Usar expressões sem realizar o fit novamente:

In [ ]:
### Usando expressões salvas
# Carregar encoders dos dados categóricos
label_encoders_local = joblib.load('./data/label_encoders_fit_com_todos_features.pkl')

# Função para avaliar a expressão com novos dados
def predict_com_expressao_local(X1, X2, X3, X4, X5, X6, X7, expression):
    variables = {
        'X1': X1, 'X2': X2, 'X3': X3, 
        'X4': X4, 'X5': X5, 'X6': X6, 'X7': X7
    }
    print()
    return eval(expression, {}, variables)

fit_com_todos_features = "(20.213402 + (0.000407 * ((((((((0.914883 * X3) + (4.488519 * X4)) + (((0.914883 * X3) + 1.628870) + (5.415799 * X4))) / (((0.122423 * X1) * 0.602120) - (-3.545621))) / 0.160798) - ((1.772454 * X6) / ((0.364922 * X5) - 0.838464))) - ((-1.024453) * X1)) + (((((((0.914883 * X3) + (4.488519 * X5)) + (((-0.628178) * X3) - 1.033868)) + (((-0.701242) * X6) * 0.838464)) / 0.160798) / (-0.073258)) + (((-0.701242) * X6) * ((0.180806 * X6) / 0.410039))))))"  # (sua expressão completa)
# Outro fit
# fit_com_todos_features = "((-6.452647) + ((-1.299273) * ((((0.498144 * X6) - ((-0.713614) * X7)) - (((-1.138196) * X3) - (((((-1.136397) * X3) - (0.205154 * (0.707107 * X4))) + ((2.986175 * X3) * 0.138477)) / (((((2.898356 * X3) * 0.138477) + (1.778016 * X5)) - ((((-1.138196) * X3) - ((-1.916468) * X5)) / ((0.096521 * X3) * (-0.013440)))) * (-0.013440))))) / (((((2.514376 * X3) * 0.138477) + (4.130474 * X5)) + (9.464716 * X5)) - ((((-1.136397) * X3) - ((0.096521 * X3) * (-0.395446))) / ((0.096521 * X3) * (-0.013440)))))))"
expression_fit_select = fit_com_todos_features

# Exemplo de uso:
dados_novo_pesquisador = {
    "academic_title": "Ph.D.",
    "institution": "Iowa State University",
    "year": 1975,
    "country": "United States",
    "num_advisors": 2,
    "num_siblings": 98,
    "generation": 31
    }

researcher_new = transformar_inputs_categoricos(label_encoders_local, dados_novo_pesquisador)
print("Dados de entrada:", researcher_new)
researcher_fortran = np.asfortranarray([[researcher_new[col] for col in features]], dtype=np.float64)

prediction = predict_com_expressao_local(768, 1073, 1975, 126, 2, 98, 31, expression_fit_select)  # Valores na ordem das features originais

print("Pesquisador exemplo deve ter:", max(1, round(prediction)), "aluno(s)")

### Encontrando a expressão por meio da regressão símbolica

In [ ]:
# Clonar df_optimized para manipular
df_predicted_students = df_optimized.copy()

### Codificar variáveis categóricas
label_encoders = {}
for col in ["academic_title", "institution", "country"]:
    le = LabelEncoder()
    df_predicted_students[col] = le.fit_transform(df_predicted_students[col].str.strip().str.lower().str.replace(" ", ""))
    label_encoders[col] = le

### Normalizar valores numéricos
#scaler = MinMaxScaler()
#df_predicted_students[features] = scaler.fit_transform(df_predicted_students[features])

### Converter em Fortran-order
features_df = np.asfortranarray(df_predicted_students[features].values, dtype=np.float64)
target_df = np.asfortranarray(df_predicted_students[target].values, dtype=np.float64)

# Criar o regressor simbólico
# model = SymbolicRegressor(
#     allowed_symbols='add,sub,mul,aq,sin,variable',
#     offspring_generator='basic',
#     optimizer_iterations=10,
#     max_length=50,
#     initialization_method='btc',
#     n_threads=32,
#     objectives = ['poisson', 'length'],
#     epsilon = 0,
#     random_state=None,
#     reinserter='keep-best',
#     max_evaluations=int(1e6),
#     symbolic_mode=False,
#     tournament_size=3
# )
model = SymbolicRegressor(
    n_threads=32
)

### Treinar o modelo com os dados
model.fit(features_df, target_df)

## Exibir o melhor modelo encontrado
print("Melhor expressão encontrado:\n", model.pareto_front_[0]['model'], "\n")

In [ ]:
# Salvar os encoders
#joblib.dump(label_encoders, './data/label_encoders.pkl')
#print("label_encoders desse treinamento salvo localmente.")

In [ ]:
### Fazer previsões
df_predicted_students["predicted_students"] = model.predict(np.asfortranarray(df_predicted_students[features].values, dtype=np.float64))

### Visualizar algumas previsões
print(df_predicted_students[[target, "predicted_students"]].head(20))

### Calcular o erro absoluto
mae = mean_absolute_error(df_predicted_students[target], df_predicted_students["predicted_students"])
print("\nErro médio absoluto:", mae)

In [ ]:
# Visualizar os resultados
mask = df_predicted_students["predicted_students"] >= 0
plt.scatter(df_predicted_students["generation"], df_predicted_students[target], label='Dados Reais')
plt.scatter(df_predicted_students["generation"][mask], df_predicted_students["predicted_students"][mask], color='red', label='Previsões do Pyperon')
plt.xlabel('Geração')
plt.ylabel('Qtd de filhos acadêmicos')
plt.legend()
plt.title('Distribuição de filhos acadêmicos por geração')
plt.show()

In [ ]:
# Visualizar os resultados
mask = (df_predicted_students["year"] > 500) & (df_predicted_students["year"] < 2100)
plt.scatter(df_predicted_students["year"][mask], df_predicted_students[target][mask], label='Dados Reais')
plt.scatter(df_predicted_students["year"][mask], df_predicted_students["predicted_students"][mask], color='red', label='Previsões do Pyperon')
plt.xlabel('Ano')
plt.ylabel('Qtd de filhos acadêmicos')
plt.legend()
plt.title('Distribuição de filhos acadêmicos por ano')
plt.show()

In [ ]:
# Reverter os valores codificados para os nomes originais
for col in ["academic_title", "institution", "country"]:
    le = label_encoders[col]
    df_predicted_students[col] = le.inverse_transform(df_predicted_students[col])

In [ ]:
print("Predição com mais alunos:\n\n", df_predicted_students[['researcher_id'] + [f for f in features if f not in ['year']] + ["predicted_students"]].nlargest(20, "predicted_students").to_string(index=False))

In [ ]:
print("Predição com menos alunos:\n\n", df_predicted_students[['researcher_id'] + [f for f in features if f not in ['year']] + ["predicted_students"]].nsmallest(20, "predicted_students").to_string(index=False))

In [ ]:
comparar_ids = [1, 4, 93526, 261324, 17394, 321123]
resultado_comparacao = df_predicted_students[df_predicted_students['researcher_id'].isin(comparar_ids)]
print(resultado_comparacao.to_string(index=False))

In [ ]:
# Perguntar sobre um novo pesquisador
dados_novo_pesquisador = {
    "academic_title": "Ph.D.",
    "institution": "Iowa State University",
    "year": 1975,
    "country": "United States",
    "num_advisors": 2,
    "num_siblings": 98,
    "generation": 31
    }

researcher_new = transformar_inputs_categoricos(label_encoders, dados_novo_pesquisador)

print("Dados de entrada:", researcher_new)
researcher_fortran = np.asfortranarray([[researcher_new[col] for col in features]], dtype=np.float64)

print("Pesquisador exemplo deve ter:", max(1, round(model.predict(researcher_fortran)[0])), "aluno(s)")

## A similaridade temática entre títulos de dissertações pode ser utilizada para prever ou recomendar potenciais orientadores?

### Verificar os dados

In [ ]:
#### Esta celula é apenas para verificar os dados, não precisa rodar ###

# Escolher banco para manipular
df_similaridade = df_optimized.copy()

# Criar embeddings dos orientadores
print("\n\nTamanho do banco com dissertações nulas:", len(df_similaridade))
df_similaridade.dropna(subset=["dissertation"])
print("Novo tamanho do banco sem dissertações nulas:", len(df_similaridade))

print("\n\nVerificando dados do banco:\n")
comparar_ids = [1, 4, 94153]
resultado_comparacao = df_similaridade[df_similaridade['researcher_id'].isin(comparar_ids)]
print(resultado_comparacao.to_string(index=False))

### Excução da regressão símbolica

In [ ]:
# Escolher banco para manipular
df_similaridade = df_optimized.copy()

# Carregar modelo BERT
bert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Criar embeddings dos orientadores
print("\n\nTamanho do banco com valores nulos:", len(df_similaridade))
df_similaridade = df_similaridade.dropna(subset=["dissertation"])
print("Novo tamanho do banco sem valores nulos:", len(df_similaridade), "\n\n")

# Gerar embeddings
orientadores_emb = {}
for _, row in tqdm(df_similaridade.iterrows(), desc="Gerando Embeddings", total=len(df_similaridade)):
    embedding = bert_model.encode(row["dissertation"])
    orientadores_emb[row["researcher_id"]] = embedding
    
    # Mostrar informações dos 2 primeiros embeddings
    if len(orientadores_emb) <= 2:
        print(f"\n=== Embedding para Researcher ID: {row['researcher_id']}")
        print(f"Título: {row['dissertation']}")
        print(f"Tamanho do embedding: {len(embedding)}")
        print(f"Primeiros 6 valores do embedding: {embedding[:6]}\n")

In [ ]:
# Lista de (id, embedding)
lista_orientadores = list(orientadores_emb.items())

# Amostrar pares aluno-orientador
pares = []
targets = []

# Pode-se controlar o número de pares para evitar explosão combinatória
n_amostras = 500000  # ajuste conforme sua máquina

for _ in tqdm(range(n_amostras), desc="Gerando pares"):
    while True:
        aluno_id, emb_aluno = random.choice(lista_orientadores)
        orientador_id, emb_orientador = random.choice(lista_orientadores)
        if aluno_id != orientador_id:
            break

    # Normalizar embeddings
    emb_aluno = normalize(emb_aluno.reshape(1, -1))[0]
    emb_orientador = normalize(emb_orientador.reshape(1, -1))[0]
    # Combinação dos embeddings: diferença absoluta (ou concatenação, soma, etc.)
    #entrada = np.abs(emb_aluno - emb_orientador)
    entrada = np.concatenate([emb_aluno, emb_orientador, np.abs(emb_aluno - emb_orientador)])


    # Similaridade cosseno entre os dois
    sim = cosine_similarity([emb_aluno], [emb_orientador])[0, 0]

    pares.append(entrada)
    targets.append(sim)

X_train = np.array(pares)
y_train = np.array(targets)

model = SymbolicRegressor(
    n_threads=32
)


print("\n\nTreinando modelo com pares...")
model.fit(X_train, y_train)
print("Treinamento concluído. \n\n")

def melhor_orientador(nova_dissertacao):
    nova_emb = bert_model.encode(nova_dissertacao)
    nova_emb = normalize(nova_emb.reshape(1, -1))[0]  # Normaliza novo embedding

    melhor_nome = None
    maior_sim = -float('inf')

    for nome, emb_orientador in orientadores_emb.items():
        emb_orientador = normalize(emb_orientador.reshape(1, -1))[0]  # Normaliza o orientador

        #entrada_modelo = np.abs(nova_emb - emb_orientador).reshape(1, -1)
        entrada_modelo = np.concatenate([nova_emb, emb_orientador, np.abs(nova_emb - emb_orientador)]).reshape(1, -1)
        sim_prevista = model.predict(entrada_modelo)[0]

        if sim_prevista > maior_sim:
            maior_sim = sim_prevista
            melhor_nome = nome

    return melhor_nome, maior_sim

print("Expressão encontrada para predizer dissertação similar:\n", model.pareto_front_[0]['model'])

plt.hist(y_train, bins=50)
plt.title("Distribuição de Similaridades")
plt.xlabel("Similaridade de Cosseno", labelpad=10)
plt.ylabel("Frequência", labelpad=10)
plt.show()

### Encontrando um orientador

In [ ]:
# Testando com uma nova dissertação
#nova_dissertacao = "General Theory Relating to the Implementation of Concurrent Symbolic Computation"
# Pega uma dissertação aleatória
nova_dissertacao = df_similaridade.dropna(subset=["dissertation"]).sample(1).iloc[0]["dissertation"]
print("Procurando similaridade para a dissertação:\n", nova_dissertacao)

melhor, score = melhor_orientador(nova_dissertacao)

melhor_researcher = df_similaridade.iloc[melhor]
print(f"\n🚀 Orientador com dissertação tendo maior similaridade 🚀\nID MGP: {melhor_researcher['researcher_id']}\nNome: {melhor_researcher['researcher_name']}\nDissertação: {melhor_researcher['dissertation']} (similaridade da predição: {score:.2%} | similaridade cosseno: {cosine_similarity(bert_model.encode(nova_dissertacao).reshape(1, -1), bert_model.encode(melhor_researcher['dissertation']).reshape(1, -1))[0, 0]:.2%})\nAno do título título acadêmico do orientador: {melhor_researcher['year']}")

## Qual é o grau de similaridade temática entre a dissertação de um pesquisador e a de seus alunos?

In [ ]:
### Gerando a lista de alunos dos orientadores
df_similaridade_alunos = df_optimized.copy()

alunos_map = defaultdict(list)

for _, row in df_similaridade_alunos.iterrows():
    aluno_id = row["researcher_id"]
    orientadores = row["advisor_ids"]

    if isinstance(orientadores, str):  # Caso esteja como string
        orientadores = [int(o.strip()) for o in orientadores.split(",") if o.strip().isdigit()]
    
    for orientador_id in orientadores:
        alunos_map[orientador_id].append(aluno_id)
print(alunos_map['17394'])

In [ ]:
def similaridade_com_alunos(orientador_id):
    orientador_id = str(orientador_id)  # 🔁 converte para string para acessar alunos_map
    
    if int(orientador_id) not in df_similaridade_alunos["researcher_id"].values:
        #print("ID de orientador não encontrado.")
        return []

    orientador_row = df_similaridade_alunos[df_similaridade_alunos["researcher_id"] == int(orientador_id)].iloc[0]
    dissert_orientador = orientador_row["dissertation"]

    if pd.isna(dissert_orientador):
        #print("Dissertação do orientador ausente.")
        return []

    emb_orientador = bert_model.encode(dissert_orientador).reshape(1, -1)

    orientandos_ids = alunos_map.get(orientador_id, [])
    if not orientandos_ids:
        #print("Este orientador não possui alunos registrados.")
        return []

    resultados = []
    for aluno_id in orientandos_ids:
        aluno_row = df_similaridade_alunos[df_similaridade_alunos["researcher_id"] == aluno_id]
        if aluno_row.empty:
            continue

        aluno_row = aluno_row.iloc[0]
        dissert_aluno = aluno_row["dissertation"]

        if pd.isna(dissert_aluno):
            continue

        emb_aluno = bert_model.encode(dissert_aluno).reshape(1, -1)
        sim_cos = cosine_similarity(emb_orientador, emb_aluno)[0, 0]

        resultados.append({
            "aluno_id": aluno_id,
            "aluno_nome": aluno_row["researcher_name"],
            "ano_aluno": aluno_row["year"],
            "similaridade_cosseno": sim_cos,
        })

    return sorted(resultados, key=lambda x: x["similaridade_cosseno"], reverse=True)


In [ ]:
resultados = similaridade_com_alunos(orientador_id=17394)

if resultados:
    media_sim = sum(r["similaridade_cosseno"] for r in resultados) / len(resultados)
    print(f"\n📊 Média de similaridade cosseno: {media_sim:.2%}")

    for r in resultados:
        print(f"\n👨‍🎓 Aluno: {r['aluno_nome']} ({r['aluno_id']}) - Ano: {r['ano_aluno']}")
        print(f"  - Similaridade cosseno: {r['similaridade_cosseno']:.2%}")

In [ ]:
resultados = similaridade_com_alunos(orientador_id=9)

if resultados:
    media_sim = sum(r["similaridade_cosseno"] for r in resultados) / len(resultados)
    print(f"\n📊 Média de similaridade cosseno: {media_sim:.2%}")

    for r in resultados:
        print(f"\n👨‍🎓 Aluno: {r['aluno_nome']} ({r['aluno_id']}) - Ano: {r['ano_aluno']}")
        print(f"  - Similaridade cosseno: {r['similaridade_cosseno']:.2%}")


In [ ]:
df_similaridade_alunos["sim_cosseno_alunos"] = None

for orientador_id in alunos_map:
    resultados = similaridade_com_alunos(int(orientador_id))

    if resultados:
        media_sim = sum(r["similaridade_cosseno"] for r in resultados) / len(resultados)
        df_similaridade_alunos.loc[df_similaridade_alunos["researcher_id"] == int(orientador_id), "sim_cosseno_alunos"] = media_sim

## Salvando os novos dados
df_similaridade_alunos.to_feather("./data/mgp_final.feather")

In [ ]:
## Lendo o novo dataframe
df_final = pd.read_feather("./data/mgp_final.feather")
df_final

In [ ]:
#FIM